In [1]:
import numpy as np
import pandas as pd

#读取文件
parser = lambda date: pd.to_datetime(date, format='%d.%m.%Y')
train = pd.read_csv('/Users/ynshan/Desktop/predict/sales_train.csv', parse_dates=['date'],
                    date_parser=parser)
test = pd.read_csv('/Users/ynshan/Desktop/predict/test.csv')
submission = pd.read_csv('/Users/ynshan/Desktop/predict/sample_submission.csv')
items = pd.read_csv('/Users/ynshan/Desktop/predict/items.csv')
item_cats = pd.read_csv('/Users/ynshan/Desktop/predict/item_categories.csv')
shops = pd.read_csv('/Users/ynshan/Desktop/predict/shops.csv')

In [3]:

#查看信息
print('train:', train.shape, 'test:', test.shape, 'items:', items.shape, 'shops:', shops.shape,'cats:',item_cats.shape)


train: (2935849, 6) test: (214200, 3) items: (22170, 3) shops: (60, 2) cats: (84, 2)


'\ntest_shops = test.shop_id.unique()\ntrain = train[train.shop_id.isin(test_shops)]\ntest_items = test.item_id.unique()\ntrain = train[train.item_id.isin(test_items)]\n'

In [4]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [5]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [6]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
item_cats.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [8]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [9]:
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month

train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month
0,2013-01-02,0,59,22154,999.00,1.0,2013,1
1,2013-01-03,0,25,2552,899.00,1.0,2013,1
2,2013-01-05,0,25,2552,899.00,-1.0,2013,1
3,2013-01-06,0,25,2554,1709.05,1.0,2013,1
4,2013-01-15,0,25,2555,1099.00,1.0,2013,1


In [11]:
#一共34个月份
train['date_block_num'].max()

33

In [10]:
# group by 
train_grp = train.groupby(['date_block_num','shop_id','item_id'])

In [11]:
# price mean by month
train_price = pd.DataFrame(train_grp.mean()['item_price']).reset_index()
train_price.head()

,date_block_num,shop_id,item_id,item_price
0,0,2,33,499.0
1,0,2,482,3300.0
2,0,2,491,600.0
3,0,2,839,3300.0
4,0,2,1007,449.0


In [12]:
# count summary by month
train_monthly = pd.DataFrame(train_grp.sum()['item_cnt_day']).reset_index()
train_monthly.rename(columns={'item_cnt_day':'item_cnt'}, inplace=True)
train_monthly.head()

,date_block_num,shop_id,item_id,item_cnt
0,0,2,33,1.0
1,0,2,482,1.0
2,0,2,491,1.0
3,0,2,839,1.0
4,0,2,1007,3.0


In [13]:

print(train_monthly['item_cnt'].describe())
print('\n[item_cnt > 20] per month：{}'.format(len(train_monthly.query('item_cnt > 20')) / 34)) 

count    600159.000000
mean          2.695571
std          11.289615
min          -4.000000
25%           1.000000
50%           1.000000
75%           2.000000
max        2253.000000
Name: item_cnt, dtype: float64

[item_cnt > 20] per month：178.76470588235293


In [14]:
train_piv = train_monthly.pivot_table(index=['item_id','shop_id'], columns=['date_block_num'], values='item_cnt', aggfunc=np.sum, fill_value=0)
train_piv = train_piv.reset_index()
train_piv.head()

date_block_num,item_id,shop_id,0,1,2,3,4,5,6,7,...,24,25,26,27,28,29,30,31,32,33
0,30,2,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,3,0,4,5,2,2,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,30,4,0,7,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,30,5,0,5,7,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,30,6,0,28,10,4,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0


In [15]:
test_only = test[~test['item_id'].isin(train['item_id'].unique())]['item_id'].unique()
len(test_only)

384

In [16]:
# By shop
grp = train_monthly.groupby(['shop_id', 'item_id'])
train_shop = grp.agg({'item_cnt':['mean','sum','median','min','max']}).reset_index()
train_shop.columns = ['shop_id','item_id','cnt_mean_shop','cnt_sum_shop','cnt_med_shop','cnt_min_shop','cnt_max_shop']
train_shop.head()

,shop_id,item_id,cnt_mean_shop,cnt_sum_shop,cnt_med_shop,cnt_min_shop,cnt_max_shop
0,2,30,1.000,4.0,1.0,1.0,1.0
1,2,31,1.600,8.0,1.0,1.0,4.0
2,2,32,1.375,11.0,1.0,1.0,2.0
3,2,33,1.100,11.0,1.0,1.0,2.0
4,2,53,1.000,1.0,1.0,1.0,1.0


In [17]:
# By shop&category
train_cat_monthly = pd.merge(train_monthly, items, on=['item_id'], how='left')
grp = train_cat_monthly.groupby(['shop_id', 'item_category_id'])
train_shop_cat = grp.agg({'item_cnt':['sum','median']}).reset_index()
train_shop_cat.columns = ['shop_id','item_category_id','cnt_sum_cat_shop','cnt_med_cat_shop']
train_shop_cat.head()

,shop_id,item_category_id,cnt_sum_cat_shop,cnt_med_cat_shop
0,2,2,141.0,1.0
1,2,3,405.0,2.0
2,2,5,16.0,1.5
3,2,6,260.0,1.0
4,2,7,76.0,1.0


In [18]:
# Last month
train_last = train_monthly[train_monthly['date_block_num'] == 33]
train_last = train_last.drop(['date_block_num'], axis=1).rename(columns={'item_cnt':'cnt_sum_last'})
train_last.head()

,shop_id,item_id,cnt_sum_last
571479,2,31,1.0
571480,2,486,3.0
571481,2,787,1.0
571482,2,794,1.0
571483,2,968,1.0


In [19]:
# Prev month
train_prev = train_monthly.copy()
train_prev['date_block_num'] = train_prev['date_block_num'] + 1
train_prev = train_prev.rename(columns={'item_cnt':'cnt_sum_prev'})
train_prev.head()

,date_block_num,shop_id,item_id,cnt_sum_prev
0,1,2,33,1.0
1,1,2,482,1.0
2,1,2,491,1.0
3,1,2,839,1.0
4,1,2,1007,3.0


In [20]:
# Prev month by category
train_cat_prev = pd.merge(train_prev, items, on=['item_id'], how='left')
grp = train_cat_prev.groupby(['date_block_num','item_category_id'])
train_cat_prev = grp['cnt_sum_prev'].sum().reset_index()
train_cat_prev = train_cat_prev.rename(columns={'cnt_sum_prev':'cnt_sum_cat_prev'})
train_cat_prev.head()

,date_block_num,item_category_id,cnt_sum_cat_prev
0,1,2,293.0
1,1,5,90.0
2,1,6,429.0
3,1,9,145.0
4,1,19,1013.0


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
item_cats

,item_category_name,item_category_id,item_category_name_len,item_category_name_wc,item_category_name_tfidf_0,item_category_name_tfidf_1,item_category_name_tfidf_2,item_category_name_tfidf_3,item_category_name_tfidf_4,item_category_name_tfidf_5,item_category_name_tfidf_6,item_category_name_tfidf_7,item_category_name_tfidf_8,item_category_name_tfidf_9,item_category_name_tfidf_10
0,PC - Гарнитуры/Наушники,0,23,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,Аксессуары - PS2,1,16,3,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,Аксессуары - PS3,2,16,3,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Аксессуары - PS4,3,16,3,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,Аксессуары - PSP,4,16,3,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,Аксессуары - PSVita,5,19,3,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,Аксессуары - XBOX 360,6,21,4,0.732955,0.680277,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,Аксессуары - XBOX ONE,7,21,4,0.732955,0.680277,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,Билеты (Цифра),8,14,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000
9,Доставка товара,9,15,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [22]:
item_cats['item_category_name_len'] = item_cats['item_category_name'].apply(len)
item_cats['item_category_name_wc'] = item_cats['item_category_name'].apply(lambda x: len(str(x).split(' ')))

feature_cnt = 11
tfidf = TfidfVectorizer(max_df=0.6, max_features=feature_cnt, ngram_range=(1, 2))
txtFeatures = pd.DataFrame(tfidf.fit_transform(item_cats['item_category_name']).toarray())
cols = txtFeatures.columns
for i in range(feature_cnt):
    item_cats['item_category_name_tfidf_' + str(i)] = txtFeatures[cols[i]]

In [23]:
#txtFeatures

In [24]:
feature_cnt = 25
tfidf = TfidfVectorizer(max_df=0.6, max_features=feature_cnt, ngram_range=(1, 2))
txtFeatures = pd.DataFrame(tfidf.fit_transform(items['item_name']).toarray())
cols = txtFeatures.columns
for i in range(feature_cnt):
    items['item_name_tfidf_' + str(i)] = txtFeatures[cols[i]]

In [25]:
shops['shop_name_len'] = shops['shop_name'].apply(len)
shops['shop_name_wc'] = shops['shop_name'].apply(lambda x: len(str(x).split(' ')))

shops['city'] = shops.shop_name.apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')[0])
shops['city'] = pd.Categorical(shops['city']).codes

In [26]:
#shops

In [27]:
def margeFeature(source): 
  d = source
  d = pd.merge(d, items, on=['item_id'], how='left')
  d = pd.merge(d, item_cats, on=['item_category_id'], how='left')
  d = pd.merge(d, shops, on=['shop_id'], how='left')

  d = pd.merge(d, train_shop, on=['shop_id','item_id'], how='left')
  d = pd.merge(d, train_shop_cat, on=['shop_id','item_category_id'], how='left')
  d = pd.merge(d, train_last, on=['shop_id','item_id'], how='left')
  #d = pd.merge(d, train_piv, on=['shop_id','item_id'], how='left')
  #d = pd.merge(d, train_prev, on=['date_block_num','shop_id','item_id'], how='left')
  #d = pd.merge(d, train_cat_prev, on=['date_block_num','item_category_id'], how='left')
  d = pd.merge(d, train_price, on=['date_block_num','shop_id','item_id'], how='left')
  
  #d.drop(['shop_id','shop_name','item_id','item_name','item_category_name'], axis=1, inplace=True)
  d.drop(['shop_name','item_name','item_category_name'], axis=1, inplace=True)
  d.fillna(0.0, inplace=True)
  return d

In [28]:
train_set = margeFeature(train_monthly)

X_train = train_set.drop(['item_cnt'], axis=1)
Y_train = train_set['item_cnt']

X_train.head()

,date_block_num,shop_id,item_id,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,item_name_tfidf_4,item_name_tfidf_5,...,city,cnt_mean_shop,cnt_sum_shop,cnt_med_shop,cnt_min_shop,cnt_max_shop,cnt_sum_cat_shop,cnt_med_cat_shop,cnt_sum_last,item_price
0,0,2,33,37,0.000000,0.0,1.0,0.0,0.0,0.0,...,0,1.100000,11.0,1.0,1.0,2.0,416.0,1.0,0.0,499.0
1,0,2,482,73,0.817274,0.0,0.0,0.0,0.0,0.0,...,0,1.400000,28.0,1.0,1.0,4.0,98.0,1.0,0.0,3300.0
2,0,2,491,73,1.000000,0.0,0.0,0.0,0.0,0.0,...,0,1.111111,10.0,1.0,1.0,2.0,98.0,1.0,0.0,600.0
3,0,2,839,73,1.000000,0.0,0.0,0.0,0.0,0.0,...,0,1.111111,10.0,1.0,1.0,2.0,98.0,1.0,0.0,3300.0
4,0,2,1007,67,0.000000,0.0,0.0,0.0,0.0,0.0,...,0,1.428571,10.0,1.0,1.0,3.0,148.0,1.0,0.0,449.0


In [29]:
test['date_block_num'] = 34

X_test = margeFeature(test.drop(['ID'], axis=1))
X_test.head()

,shop_id,item_id,date_block_num,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,item_name_tfidf_4,item_name_tfidf_5,...,city,cnt_mean_shop,cnt_sum_shop,cnt_med_shop,cnt_min_shop,cnt_max_shop,cnt_sum_cat_shop,cnt_med_cat_shop,cnt_sum_last,item_price
0,5,5037,34,19,0.0,0.000000,0.0,0.0,0.0,0.0,...,3,1.444444,13.0,1.0,1.0,3.0,1901.0,1.0,0.0,0.0
1,5,5320,34,55,0.0,0.000000,0.0,0.0,0.0,0.0,...,3,0.000000,0.0,0.0,0.0,0.0,3276.0,1.0,0.0,0.0
2,5,5233,34,19,0.0,0.000000,0.0,0.0,0.0,0.0,...,3,2.000000,10.0,2.0,1.0,3.0,1901.0,1.0,1.0,0.0
3,5,5232,34,23,0.0,0.526615,0.0,0.0,0.0,0.0,...,3,1.000000,1.0,1.0,1.0,1.0,1346.0,1.0,0.0,0.0
4,5,5268,34,20,0.0,0.000000,0.0,0.0,0.0,0.0,...,3,0.000000,0.0,0.0,0.0,0.0,1427.0,2.0,0.0,0.0


In [30]:
#test

In [31]:
from sklearn import ensemble, metrics
from sklearn import linear_model, preprocessing

In [32]:
corr = train_set.corr()['item_cnt']
corr_imp_20 = corr.abs().sort_values(ascending=False)[1:20]
corr_imp_20

cnt_mean_shop                  0.798488
cnt_med_shop                   0.779737
cnt_sum_shop                   0.772971
cnt_med_cat_shop               0.751201
cnt_max_shop                   0.634712
cnt_sum_last                   0.385293
cnt_min_shop                   0.324948
item_name_tfidf_0              0.166238
cnt_sum_cat_shop               0.069552
item_category_name_tfidf_7     0.058252
item_category_name_tfidf_4     0.043464
item_category_name_wc          0.041931
item_name_tfidf_10             0.040852
item_name_tfidf_20             0.039025
item_category_name_tfidf_8     0.037462
item_name_tfidf_15             0.033404
item_name_tfidf_19             0.033299
date_block_num                 0.030523
item_category_name_tfidf_10    0.030408
Name: item_cnt, dtype: float64

In [33]:
_ = '''
col = [c for c in train_set.columns if c not in ['item_cnt']]
x1 = train_set[train_set['date_block_num'] != 33]
y1 = np.log1p(x1['item_cnt'].clip(0.,20.))
x1 = x1[col]

x2 = train_set[train_set['date_block_num'] == 33]
y2 = np.log1p(x2['item_cnt'].clip(0.,20.))
x2 = x2[col]

reg = ensemble.ExtraTreesRegressor(n_estimators=25, n_jobs=-1, max_depth=15, random_state=42)

reg.fit(x1,y1)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y2.clip(0.,20.), reg.predict(x2).clip(0.,20.))))

feature_imp = pd.DataFrame(reg.feature_importances_, index=col, columns=["importance"])
feat_imp_20 = feature_imp.sort_values("importance", ascending=False).head(20)
feat_imp_20
'''

In [34]:
_ = '''
reg = ensemble.ExtraTreesRegressor(n_estimators=25, n_jobs=-1, max_depth=15, random_state=42)

reg.fit(X_train, Y_train)
pred_cnt = reg.predict(X_test)
'''

In [35]:
_ = '''
sc = preprocessing.StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

reg = linear_model.ElasticNet(alpha=1.0, l1_ratio=0.5)
#reg = linear_model.Lasso(alpha=1.0)

reg.fit(X_train, Y_train)
pred_cnt = reg.predict(X_test)
'''

In [48]:
import xgboost as xgb

sc = preprocessing.StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

reg = xgb.XGBRegressor(n_estimators=25, max_depth=12, learning_rate=0.1, subsample=1, colsample_bytree=1, eval_metric='rmse')
#reg = xgb.XGBRegressor(n_estimators=360, max_depth=3, learning_rate=0.1, subsample=1, colsample_bytree=1, eval_metric='rmse')

reg.fit(X_train, Y_train)
pred_cnt = reg.predict(X_test)


In [49]:
result = pd.DataFrame({
    "ID": test["ID"],
    "item_cnt_month": pred_cnt.clip(0. ,20.)
})
result.to_csv("submission.csv", index=False)

In [50]:
print(len(pred_cnt[pred_cnt > 20]))
result.head(30)

563


,ID,item_cnt_month
0,0,1.553646
1,1,0.080145
2,2,1.966785
3,3,0.964122
4,4,0.122494
5,5,1.853408
6,6,2.605494
7,7,2.133590
8,8,9.696458
9,9,0.080145


In [54]:
result.to_csv("submission.csv", index=False)